## Resegment the data using Xenium Ranger

[Xenium Ranger documentation for re-segmentation](https://www.10xgenomics.com/support/software/xenium-ranger/latest/analysis/running-pipelines/XR-resegment)

Used version 1.7.1.1 here

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_csv, concat
from seaborn import barplot
from os.path import exists

import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

%matplotlib inline
# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#### set notebook variables

In [ ]:
# naming
project = 'mouse_snca_spatial'
platform = 'xenium'

# directories
wrk_dir = '/labshare/LNG_spatial_transcriptomics/mouse_350plex_custom_Feb2024'
xenium_samples_dir = f'{wrk_dir}/20240215__174229__FEB2024_LNG_100plex'
xenium_ranger_path = '/home/gibbsr/xeniumranger-xenium1.7/bin/xeniumranger'

# out files

# variables
DEBUG = True
slides = ['0028089__ntg', '0028090__tg']
sections = list(range(1, 4+1))
slide_samples_frmt = {
    '0028089__ntg': 'Line61nTg_{section_num}',
    '0028090__tg': 'Line61Tg_{section_num}'
}
slide_name_frmt = 'output-XETG00175__{slide_id}_{section_num}__20240215__174342'
expansion_distance = 5
CPU_CNT = 12
MAX_MEM = 64

### format Xenium Ranger re-segement commands

In [ ]:
%%time
reseg_cmds = []
for slide in slides:
    for section in sections:
        slide_name = slide_name_frmt.format(slide_id=slide, section_num=section)
        slide_path = f'{xenium_samples_dir}/{slide_name}'
        sample_name = slide_samples_frmt.get(slide).format(section_num=section)
        this_cmd = f'nohup {xenium_ranger_path} resegment --id={sample_name} \
--xenium-bundle={slide_path} --expansion-distance={expansion_distance} \
--resegment-nuclei=false --localcores {CPU_CNT} --localmem {MAX_MEM} --localvmem {MAX_MEM} \
--disable-ui true > {sample_name}_resegment.log &'
        reseg_cmds.append(this_cmd)
        
if DEBUG:
    display(reseg_cmds)

#### run the commands at the terminal

In [ ]:
print('### run these commands at the terminal')
print(f'cd {wrk_dir}/xenium_{expansion_distance}_micron_expansion')
print('#')
for cmd in reseg_cmds:
    print(cmd)

### take a look at the re-segmented summary metrics
load the other expansion sizes as well

In [ ]:
%%time
metrics_df = None
for slide in slides:
    for section in sections:
        for micron_size in [15, 5, 0]:
            slide_name = slide_name_frmt.format(slide_id=slide, section_num=section)
            slide_path = f'{xenium_samples_dir}/{slide_name}'
            sample_name = slide_samples_frmt.get(slide).format(section_num=section)
            if micron_size == 15:
                metric_file = f'{slide_path}/metrics_summary.csv'
            else:
                metric_file = f'{wrk_dir}/xenium_{micron_size}_micron_expansion/{sample_name}/outs/metrics_summary.csv'
            if not exists(metric_file):
                continue
            this_df = read_csv(metric_file)
            this_df['slide'] = slide
            this_df['section'] = section
            this_df['sample_name'] = sample_name
            this_df['micron_size'] = micron_size
            metrics_df = concat([metrics_df, this_df])
print(metrics_df.shape)
if DEBUG:
    display(metrics_df)

In [ ]:
features = ['fraction_empty_cells', 'fraction_transcripts_assigned', 
                'median_genes_per_cell', 'median_transcripts_per_cell']
for this_feature in features:
    display(metrics_df.groupby('micron_size')[this_feature].mean())

In [ ]:
for this_feature in features:
    with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 50}):
        plt.style.use('seaborn-v0_8-bright')  
        df = metrics_df[['sample_name', this_feature, 'micron_size']].copy()
        df['percent'] = df[this_feature] * 100
        # barplot(x=df.index, y='percent', hue='type', data=df, palette='Purples')
        barplot(x='sample_name', y='percent', hue='micron_size', data=df, palette=['purple', 'grey'])
        plt.grid(axis='y')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.ylabel('percent')
        plt.title(this_feature)
        plt.show()    